**С прошлого семинара**

In [53]:
class A:
    def __init__(self):
        print("started call init A")
        super().__init__()
        print("ended call init A")

class B:
    def __init__(self):
        print("started call init B")
        super().__init__()
        print("ended call init B")

class C(A, B):
    def __init__(self):
        print("started call init C")
        super().__init__()
        print("ended call init C")

In [54]:
c = C()

started call init C
started call init A
started call init B
ended call init B
ended call init A
ended call init C


In [78]:
class A:
    def __init__(self):
        print("called init A")

class B:
    def __init__(self):
        print("called init B")

class C(A, B):
    def __init__(self):
        super().__init__()  # super(C.__mro__[0], self).__init__()
        super(A, self).__init__()

In [79]:
c = C()

called init A
called init B


# Семинар 11: декораторы и itertools

### Глава 1: декораторы

In [87]:
import typing as tp

def uppercase(func: tp.Callable[[], str]) -> tp.Callable[[], str]:  # передаем функцию, которую будем оборачивать
    def wrapper():
        result = func()
        return result.upper()
    return wrapper # вернуть нужно функцию-обертку

In [88]:
@uppercase
def print_hello():
    return "hello world"

In [89]:
print_hello()

'HELLO WORLD'

In [90]:
def invert_arguments(func):
    def wrapper(*args):
        result = func(*args[::-1])
        return result
    return wrapper

In [91]:
@invert_arguments
def print_hello(a, b, c):
    print(a, b, c)

In [92]:
print_hello(1, 2, 3)

3 2 1


In [103]:
def uppercase(func):  # передаем функцию, которую будем оборачивать
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper()
    return wrapper # вернуть нужно функцию-обертку

In [104]:
class A:
    @uppercase
    def __str__(self):
        return 'some abstract class'

    # __str__ = uppercase(__str__)

In [106]:
a = A()
print(a.__str__())
# a.__str__() <=> A.__str__(a) <=> uppercase(A.__str__)(a) <=> wrapper(a)

SOME ABSTRACT CLASS


In [113]:
def uppercase(func):  # передаем функцию, которую будем оборачивать
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper()
    return wrapper # вернуть нужно функцию-обертку

class A:
    @uppercase
    def __str__(self):
        """Converts object to str."""
        return 'some abstract class'

a = A()
print(a)  # будет ли этот декоратор работать с классами? если нет, то как поправить?

SOME ABSTRACT CLASS


In [114]:
A.__str__.__name__

'wrapper'

In [115]:
A.__str__.__doc__

In [116]:
A.__str__(a)

'SOME ABSTRACT CLASS'

In [117]:
a.__str__()

'SOME ABSTRACT CLASS'

При этом мы теряем метаданные о функции:

In [119]:
import functools


def uppercase(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper()
    return wrapper

class A:
    @uppercase
    def __str__(self):
        """Converts result to uppercase."""
        return 'some abstract class'

In [120]:
A.__str__.__name__

'__str__'

In [121]:
A.__str__.__doc__

'Converts result to uppercase.'

In [122]:
def uppercase(func):
    # @functools.wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper()

    # wrapper = functools.wraps(func)(wrapper)
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__

    return wrapper

class A:
    @uppercase
    def __str__(self):
        """Converts result to uppercase."""
        return 'some abstract class'

In [123]:
print(A.__str__.__name__)
print(A.__str__.__doc__)

__str__
Converts result to uppercase.


In [2]:
def wraps(func):
    def decorator(decorated_func):
        def inner(*args, **kwargs):
            return decorated_func(*args, **kwargs)
        inner.__name__ = func.__name__
        inner.__doc__ = func.__doc__
        return inner
    return decorator

def uppercase(func):
    # @functools.wraps(func)
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper()

    # wrapper = functools.wraps(func)(wrapper)

    # wrapper.__name__ = func.__name__
    # wrapper.__doc__ = func.__doc__

    return wrapper

class A:
    @uppercase
    def __str__(self):
        """Converts result to uppercase."""
        return 'some abstract class'

In [3]:
print(A.__str__.__name__)
print(A.__str__.__doc__)

__str__
Converts result to uppercase.


Для чего в реальной жизни удобно использовать декоратор?

Например, замерять суммарное время работы функций

In [4]:
import functools
import time


def use_timer(result_struct: dict):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            elapsed_time = time.time() - start

            result_struct.setdefault(func.__name__, 0)
            result_struct[func.__name__] += elapsed_time

            return result

        return wrapper
    return decorator


class Worker:
    times = {}

    @use_timer(times)
    def do_light(self):
        for _ in range(10000):
            ...

    @use_timer(times)
    def do_medium(self):
        for _ in range(100000):
            ...

    @use_timer(times)
    def do_hard(self):
        for _ in range(1000000):
            ...

In [5]:
w = Worker()

for _ in range(100):
    w.do_light()
    w.do_medium()
    w.do_hard()

In [6]:
w.times

{'do_light': 0.025069713592529297,
 'do_medium': 0.2478482723236084,
 'do_hard': 2.4894704818725586}

In [7]:
w.do_hard.__name__

'do_hard'

Декорировать можно также и классы

In [34]:
from functools import wraps
from dataclasses import dataclass
from uuid import uuid4


def add_id(_class):
    # @wraps(_class)
    class WithID(_class):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.__id = str(uuid4())

        @classmethod
        def __name__(cls):
            return _class.__name__

        def __str__(self):
            return super().__str__()

        def __repr__(self):
            return super().__repr__()

        def get_id(self):
            return self.__id

    return WithID


@add_id
@dataclass
class Person:
    name: str
    email: str

# Person = add_id(dataclass(Person))

In [35]:
p = Person(name="John", email="john@hse.ru")
print(p.get_id())

dafc7b0e-cf3a-44f1-ab12-36347b9781a5


In [36]:
p

add_id.<locals>.WithID(name='John', email='john@hse.ru')

Стоит учесть, что порядок декораторов важен!! Попробуйте поменять dataclass и add_id местами.

Можно еще подменять функции по ссылкам:

In [40]:
from dataclasses import dataclass
from uuid import uuid4


def add_id(_class):
    original_init = _class.__init__

    def __init__(self, *args, **kargs):
        original_init(self, *args, **kargs)
        self.id = str(uuid4())

    _class.__init__ = __init__
    return _class


@add_id
@dataclass
class Person:
    name: str
    email: str

In [41]:
p = Person(name="John", email="john@hse.ru")
print(p.id)

f60fc088-3de9-417d-a3f9-bc2fc9a5a544


In [42]:
p

Person(name='John', email='john@hse.ru')

### Глава 2: itertools

In [43]:
a = [1, -1, 6, 3, -2, 0, -6]

list(map(lambda x: x ** 2, filter(lambda x: x < 0, a)))

[1, 4, 36]

In [48]:
from functools import reduce

a = [1, 7, 3, 2, 5, 3]

reduce(lambda x, y: x + y, a, 0)

21

In [50]:
a = ["", "", "", "abc", "", "cde"]

reduce(lambda x, y: x + y, a)

'abccde'

In [51]:
import itertools

In [52]:
a = [1, 7, 3, 2, 5, 3]
list(itertools.accumulate(a))  # это как reduce с промежуточными значениями

[1, 8, 11, 13, 18, 21]

In [53]:
a = ["", "", "", "abc", "", "cde"]
list(itertools.accumulate(a))  # это как reduce с промежуточными значениями

['', '', '', 'abc', 'abc', 'abccde']

In [54]:
a = [1, 2, 3]
b = ["abc", "cde"]
c = [(1, 2), (3, 4)]

for x in itertools.chain(a, b, c):
    print(x)

1
2
3
abc
cde
(1, 2)
(3, 4)


In [55]:
a = ["abc", "cde"]

for x in itertools.chain.from_iterable(a):
    print(x)

a
b
c
c
d
e


In [57]:
a = "abcdef"
mask = [1, 0, 1, 1, 0, 0]

"".join([a[i] for i in range(len(a)) if mask[i] == 1])

'acd'

In [56]:
a = "abcdef"

"".join(itertools.compress(a, [1, 0, 1, 1, 0, 0]))

'acd'

In [28]:
a = [-1, -2, -3, 6, 2, 1, -2, -1]

list(itertools.dropwhile(lambda x: x < 0, a))

[6, 2, 1, -2, -1]

In [58]:
a = [-1, -2, -3, 6, 2, 1, -2, -1]
cond = lambda x: x < 0

while len(a):
    if cond(a[0]):
        a = a[1:]
    else:
        break

a

[6, 2, 1, -2, -1]

In [59]:
list(itertools.takewhile(lambda x: x < 0, a))

[]

In [60]:
a = [1, -1, 6, 3, -2, 0, -6]

list(itertools.filterfalse(lambda x: x < 0, a))

[1, 6, 3, 0]

In [62]:
[x for x in a if not x < 0]

[1, 6, 3, 0]

Пусть есть задача: надо сделать из строки вида "ABBBCC" строку вида "1A3B2C"

In [63]:
a = [2, -2, 2, -2, 1, -1, -1, 2, -2, 2, -2, 0, 2, 1]

for elem, group in itertools.groupby(a, key=abs):
    print(elem, "|", *group)

2 | 2 -2 2 -2
1 | 1 -1 -1
2 | 2 -2 2 -2
0 | 0
2 | 2
1 | 1


In [64]:
s = input()

# print(list(itertools.groupby(s)))

# for elem, group in itertools.groupby(s):
#     print(elem, *group)

print(
    "".join(
        f"{len(list(group))}{symbol}"
        for symbol, group in itertools.groupby(s)
    ),
)

ABBBCC
1A3B2C


In [65]:
a = [1, 2, 3, 4, 5, 6, 7]

# a[1:5:2]
for x in itertools.islice(a, 1, 5, 2):  # аналог среза только без создания массива нового
    print(x)

2
4


In [66]:
a = ["A", "T", "G", "C"]

for x in itertools.pairwise(a):
    print(x)

('A', 'T')
('T', 'G')
('G', 'C')


In [69]:
from operator import itemgetter

a = [6, 3, 7, 1, 2, 3]

print(*map(itemgetter(1), filter(lambda x: x[1] > x[0], itertools.pairwise(a))))

7 2 3


In [70]:
list(itertools.starmap(pow, [(2, 5), (3, 2), (10, 3)])) #  func(*seq[0]), func(*seq[1]) и тд

[32, 9, 1000]

In [71]:
a = ["A", "T", "G", "C"]
b = [1, 2, 3, 4]

print(list(itertools.combinations(a, 2)))

[('A', 'T'), ('A', 'G'), ('A', 'C'), ('T', 'G'), ('T', 'C'), ('G', 'C')]


In [72]:
print(list(itertools.product(a, b)))

[('A', 1), ('A', 2), ('A', 3), ('A', 4), ('T', 1), ('T', 2), ('T', 3), ('T', 4), ('G', 1), ('G', 2), ('G', 3), ('G', 4), ('C', 1), ('C', 2), ('C', 3), ('C', 4)]


In [73]:
print(list(itertools.permutations(a)))

[('A', 'T', 'G', 'C'), ('A', 'T', 'C', 'G'), ('A', 'G', 'T', 'C'), ('A', 'G', 'C', 'T'), ('A', 'C', 'T', 'G'), ('A', 'C', 'G', 'T'), ('T', 'A', 'G', 'C'), ('T', 'A', 'C', 'G'), ('T', 'G', 'A', 'C'), ('T', 'G', 'C', 'A'), ('T', 'C', 'A', 'G'), ('T', 'C', 'G', 'A'), ('G', 'A', 'T', 'C'), ('G', 'A', 'C', 'T'), ('G', 'T', 'A', 'C'), ('G', 'T', 'C', 'A'), ('G', 'C', 'A', 'T'), ('G', 'C', 'T', 'A'), ('C', 'A', 'T', 'G'), ('C', 'A', 'G', 'T'), ('C', 'T', 'A', 'G'), ('C', 'T', 'G', 'A'), ('C', 'G', 'A', 'T'), ('C', 'G', 'T', 'A')]


In [74]:
a = ["A", "T", "G", "C"]

iter = itertools.cycle(a)

print(next(iter))
print(next(iter))
print(next(iter))
print(next(iter))
print(next(iter))
print(next(iter))
print(next(iter))

A
T
G
C
A
T
G


### Зачем нужны itertools

1) Можно сделать свой спиннер

In [41]:
import itertools
import sys
import time

def spinner(seconds):
    symbols = itertools.cycle('-|/')
    tend = time.time() + seconds
    while time.time() < tend:
        sys.stdout.write('\rPlease wait... ' + next(symbols)) # no newline
        sys.stdout.flush()
        time.sleep(0.1)
    print()

if __name__ == "__main__":
    spinner(3)

Please wait... /


2) Быстро погруппировать словарь по значениям

In [42]:
from operator import itemgetter


d = {
    "a": 1,
    "b": 2,
    "c": 3,
    "d": 1,
    "e": 2,
    "f": 3,
}

for value, items in itertools.groupby(sorted(d.items(), key=itemgetter(1)), key=itemgetter(1)):
    print(value, ":", *map(itemgetter(0), items))

1 : a d
2 : b e
3 : c f


3) Выбрать из списка чисел пару (тройку, четверку) с наибольшим произведением

In [ ]:
import functools

a = list(map(int, input().split()))
n = int(input())

print(
    max(
        itertools.combinations(a, n),
        key=functools.partial(functools.reduce, lambda x, y: x * y),
    )
)

### Что дальше?

1) Есть еще more-itertools, предоставляющие уйму других возможностей: https://more-itertools.readthedocs.io/en/stable/
2) Замечательная статья как упарываться декораторами и итераторами: https://www.bbayles.com/index/decorator_factory
3) https://pydash.readthedocs.io/en/latest/